In [1]:
# bow vs tfidf

# Import necessary libraries
import mlflow
import mlflow.sklearn
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import re
import string
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
import numpy as np
import os

import dagshub

In [2]:
df = pd.read_csv('/home/madan/Machine_learning/mloops_dsmp/mlops_project/data/raw/tweet_emotions.csv').drop(columns=['tweet_id'])


In [3]:
# Define text preprocessing functions
def lemmatization(text):
    """Lemmatize the text."""
    lemmatizer = WordNetLemmatizer()
    text = text.split()
    text = [lemmatizer.lemmatize(word) for word in text]
    return " ".join(text)

def remove_stop_words(text):
    """Remove stop words from the text."""
    stop_words = set(stopwords.words("english"))
    text = [word for word in str(text).split() if word not in stop_words]
    return " ".join(text)

def removing_numbers(text):
    """Remove numbers from the text."""
    text = ''.join([char for char in text if not char.isdigit()])
    return text

def lower_case(text):
    """Convert text to lower case."""
    text = text.split()
    text = [word.lower() for word in text]
    return " ".join(text)

def removing_punctuations(text):
    """Remove punctuations from the text."""
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = text.replace('؛', "")
    text = re.sub('\s+', ' ', text).strip()
    return text

def removing_urls(text):
    """Remove URLs from the text."""
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    return url_pattern.sub(r'', text)

def normalize_text(df):
    """Normalize the text data."""
    try:
        df['content'] = df['content'].apply(lower_case)
        df['content'] = df['content'].apply(remove_stop_words)
        df['content'] = df['content'].apply(removing_numbers)
        df['content'] = df['content'].apply(removing_punctuations)
        df['content'] = df['content'].apply(removing_urls)
        df['content'] = df['content'].apply(lemmatization)
        return df
    except Exception as e:
        print(f'Error during text normalization: {e}')
        raise

# Normalize the text data
df = normalize_text(df)

x = df['sentiment'].isin(['happiness','sadness'])
df = df[x]

df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})

<>:30: SyntaxWarning: invalid escape sequence '\s'
<>:30: SyntaxWarning: invalid escape sequence '\s'
/tmp/ipykernel_24566/3090897219.py:30: SyntaxWarning: invalid escape sequence '\s'
  text = re.sub('\s+', ' ', text).strip()
/tmp/ipykernel_24566/3090897219.py:58: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'sadness':0, 'happiness':1})


In [ ]:
mlflow.set_tracking_uri("https://dagshub.com/madankh00123/Mini-mlopsProject.mlflow")

dagshub.init(repo_owner='madankh00123', repo_name='Mini-mlopsProject', mlflow=True)

mlflow.set_experiment("tfidf_vs_bow_model")

Initialized MLflow to track repo "madankh00123/Mini-mlopsProject"

Repository madankh00123/Mini-mlopsProject initialized!

2025/09/05 16:52:24 INFO mlflow.tracking.fluent: Experiment with name 'tfidf_vs_bow_model' does not exist. Creating a new experiment.


<Experiment: artifact_location='mlflow-artifacts:/93781f4828c24d98a98b17ea738af924', creation_time=1757070477280, experiment_id='6', last_update_time=1757070477280, lifecycle_stage='active', name='tfidf_vs_bow_model', tags={}>

In [10]:
vectorizers = {
    'BoW': CountVectorizer(),
    'TF-IDF': TfidfVectorizer()
}

# Define algorithms
algorithms = {
    'LogisticRegression': LogisticRegression(),
    'MultinomialNB': MultinomialNB(),
    'XGBoost': XGBClassifier(),
    'RandomForest': RandomForestClassifier(),
    'GradientBoosting': GradientBoostingClassifier()
}

In [12]:
with mlflow.start_run(run_name="All Experiments") as parent_run:
    # Loop through algorithms and feature extraction methods (Child Runs)
    for algo_name, algorithm in algorithms.items():
        for vec_name, vectorizer in vectorizers.items():
            with mlflow.start_run(run_name=f"{algo_name} with {vec_name}", nested=True) as child_run:
                X = vectorizer.fit_transform(df['content'])
                y = df['sentiment']
                X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
                

                # Log preprocessing parameters
                mlflow.log_param("vectorizer", vec_name)
                mlflow.log_param("algorithm", algo_name)
                mlflow.log_param("test_size", 0.2)
                
                # Model training
                model = algorithm
                model.fit(X_train, y_train)
                
                # Log model parameters
                if algo_name == 'LogisticRegression':
                    mlflow.log_param("C", model.C)
                elif algo_name == 'MultinomialNB':
                    mlflow.log_param("alpha", model.alpha)
                elif algo_name == 'XGBoost':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("learning_rate", model.learning_rate)
                elif algo_name == 'RandomForest':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("max_depth", model.max_depth)
                elif algo_name == 'GradientBoosting':
                    mlflow.log_param("n_estimators", model.n_estimators)
                    mlflow.log_param("learning_rate", model.learning_rate)
                    mlflow.log_param("max_depth", model.max_depth)
                
                # Model evaluation
                y_pred = model.predict(X_test)
                accuracy = accuracy_score(y_test, y_pred)
                precision = precision_score(y_test, y_pred)
                recall = recall_score(y_test, y_pred)
                f1 = f1_score(y_test, y_pred)
                
                # Log evaluation metrics
                mlflow.log_metric("accuracy", accuracy)
                mlflow.log_metric("precision", precision)
                mlflow.log_metric("recall", recall)
                mlflow.log_metric("f1_score", f1)
                
                # Log model
                import joblib
                joblib.dump(model, "model.pkl")
                mlflow.log_artifact("model.pkl")
                
                # Save and log the notebook
                notebook_path = "/home/madan/Machine_learning/mloops_dsmp/mlops_project/notebooks/exp1_baseline_model.ipynb"
                mlflow.log_artifact(notebook_path)
    
                
                # Print the results for verification
                print(f"Algorithm: {algo_name}, Feature Engineering: {vec_name}")
                print(f"Accuracy: {accuracy}")
                print(f"Precision: {precision}")
                print(f"Recall: {recall}")
                print(f"F1 Score: {f1}")

Algorithm: LogisticRegression, Feature Engineering: BoW
Accuracy: 0.7937349397590362
Precision: 0.7846750727449079
Recall: 0.7970443349753694
F1 Score: 0.7908113391984359
🏃 View run LogisticRegression with BoW at: https://dagshub.com/madankh00123/Mini-mlopsProject.mlflow/#/experiments/6/runs/e5ab76e75a2a4007a5e968e183c950ae
🧪 View experiment at: https://dagshub.com/madankh00123/Mini-mlopsProject.mlflow/#/experiments/6
Algorithm: LogisticRegression, Feature Engineering: TF-IDF
Accuracy: 0.7942168674698795
Precision: 0.777882797731569
Recall: 0.8108374384236453
F1 Score: 0.79401833092137
🏃 View run LogisticRegression with TF-IDF at: https://dagshub.com/madankh00123/Mini-mlopsProject.mlflow/#/experiments/6/runs/788eaab6e72a43368bda5a48c169911c
🧪 View experiment at: https://dagshub.com/madankh00123/Mini-mlopsProject.mlflow/#/experiments/6
Algorithm: MultinomialNB, Feature Engineering: BoW
Accuracy: 0.7826506024096386
Precision: 0.7797619047619048
Recall: 0.774384236453202
F1 Score: 0.77706